# Lab A Introduction, Random Numbers, Power, Simulation and Machine Learning  
By Simon Van Norden and Nicolas Harvie

***
## Introduction to the Class
CoCalc Basics:
- CoCalc user interface
- Launch a project
- Time travel
- Submit assignments
- Assistance

VSCode support:
- Select the "New" tab
- Choose the button that says "VS Code Server" and launch your

My tips for the course
***

# Random Numbers in Python

## Outline

1. Introduction
2. Random Number Generators
3. Shuffling and Bootstrapping
4. Distributions
5. Useful Tricks

## Introduction

The most confusing things about random number functions in Python are
- there are so many of them.
- there is so much duplication.

That's because many different modules (like numpy, pandas, scipy, etc) have their own random number functions, in addition to the ones in basic Python. Confusingly, the names and syntax of similar commands may differ across modules. 

I'll introduce you to some of the most common functions, with a focus on commands for simulation experiments in NumPy and Pandas, and on special random distributions in Python and SciPy. 

Before I do that however, here's a quick review of how computers generate random numbers.

***

## Random Number Generators

Computers use special algorithms called random number generators (RNGs) to generate "random" numbers. Picky people call them "pseudo-random" numbers, because computers don't behave randomly; they follow precise repeatable algorithms. In the case of RNGs, however, we can't predict their output on the basis of their past outputs, so we can safely treat their output as random. 

The key to their behaviour, however, is a secret value called the ``seed``, which uniquely determines the next number that they generate. Each time they generate a random value, they update their seed as well. By default, most RNGs will use the current system time as a seed the first time that they are called, so that running a program several times will give different (seemingly random) results.

They also allow users to choose their own seed. This allows users to replicate results using the same sets of "random" numbers.

Both Python and NumPy provide modules called ``random`` that provide the backbone of most of the functions that we discuss below. Each has its own RNG with its own seed. In Python, ``random()`` returns a random number $x \: | \: 0 \le x \lt 1.$ The equivalent command in NumPy is ``rand()``.

Here's some simple examples using the RNGs in Python and in Numpy. (We'll see a Pandas example a bit later on.)

In [19]:
# First, work with standard Python
from random import seed
from random import random

My_seed_for_the_generator = 20210311

# Generate 3 random numbers between 0 and 1
print(random(), random(), random())

seed(My_seed_for_the_generator)
# Three more random numbers between 0 and 1
# but these are determined by the seed value I choose
print(random(), random(), random())

seed(19980311)
# I've reset the seed, so I should see the last 3 numbers repeated.
print(random(), random(), random())

0.29936960918079325 0.8442092016016632 0.7123730119064993
0.4318592703288652 0.3116345283033898 0.9682368229868882
0.5331786119714597 0.6621567798924323 0.17289371739475723


In [20]:
# Next, work with the NumPy functions
import numpy.random as npr

# Seeding the Python RNG
seed(My_seed_for_the_generator)
print(npr.rand(),npr.rand(),npr.rand())
# Seeding the Python RNG again
seed(My_seed_for_the_generator)
print(npr.rand(),npr.rand(),npr.rand())
# I get different results, because npr.rand() is using the NumPy RNG.

0.1428208034808467 0.9502697139752301 0.49171070977884657
0.6323377654893042 0.9797543302082113 0.37033429442894084


In [21]:
# I can get repeatable results by seeding the NumPy RNG
npr.seed(My_seed_for_the_generator)
print(npr.rand(),npr.rand(),npr.rand())
npr.seed(111)
print(npr.rand(),npr.rand(),npr.rand())

0.051244674694756465 0.2572085870185694 0.5835244513324871
0.6121701756176187 0.16906975434563642 0.4360590193711702


In [22]:
# Array of random numbers 
print( npr.rand(10) )

[0.76926247 0.2953253  0.14916296 0.02247832 0.42022449 0.23868214
 0.33765619 0.99071246 0.23772645 0.08119266]


In [23]:
from random import randint

print(randint(0,4),randint(0,4),randint(0,4),randint(0,4),randint(0,4),randint(0,4))
print(npr.randint(0,4,6))

3 0 2 2 4 1
[0 2 2 0 1 1]


### New Method

**NumPy** 1.17 introduced _BitGenerators_ and _Generators_ as the new ways to handle the same tasks.

- The old method still works \(for now\) but the new one is intended to improve control \(especially when running multiple threads in parallel.\)

The new method begins by <u>initializing</u>a _BitGenerator_, then using a _Generator_ to <u>transform</u>that BitGenerator into random values ​​from a desired distribution.

There are several ways to do this, but here is the easiest. \(See [NumPy.random](https://numpy.org/doc/stable/reference/random/generator.html)
for more examples and details.\)

In [24]:
# instead of this (legacy version)
from numpy import random
oldvals = random.standard_normal(10)
print(f"Old random numbers: \n{oldvals}")

# Do this (new version)
from numpy.random import default_rng
rng = default_rng()  # rng is now our BitGenerator
newvals = rng.standard_normal(10)   # standard_normal() is our Generator
print(f"New random numbers: \n{newvals}")

Old random numbers: 
[-1.02796733 -0.09098625  0.492003    0.4246722   1.28304882  0.31598645
 -0.4080822  -0.06794759 -0.95242666 -0.1106774 ]
New random numbers: 
[-2.10140579  1.79757271 -0.28645135 -0.08952693 -1.26324789 -1.10236862
 -1.70368915  0.04501836 -1.47522953  0.49370114]


In [25]:
print(rng)

Generator(PCG64)


This tells us what type of BitGenerator we are using.

- This allows advanced users to select alternative algorithms to generate pseudo\-random numbers.
- This might be important if you were doing crypto, for example, but you won't have to worry about it for this course. The `default_rng` will be just fine....

Suppose we now want to seed our RNG so that our results are reproducible. How do we do this using a _BitGenerator_?

In [26]:
import numpy as np
# We seed our BitGenerator
rng = np.random.default_rng(seed=42)
# Now we can draw some numbers by applying the Generator to the BitGenerator
newvals = rng.standard_normal(5)
print(f"New random numbers: \n{newvals}")
newvals = rng.standard_normal(5)
print(f"New random numbers: \n{newvals}")
# Now we can reseed the BitGenerator
rng = np.random.default_rng(seed=19980311)
newvals = rng.standard_normal(5)
print(f"New random numbers: \n{newvals}")

New random numbers: 
[ 0.30471708 -1.03998411  0.7504512   0.94056472 -1.95103519]
New random numbers: 
[-1.30217951  0.1278404  -0.31624259 -0.01680116 -0.85304393]
New random numbers: 
[-0.03703493  1.18719744 -0.15631175 -0.69874877  0.7609645 ]


In [27]:
# We can seed multiple BitGenerators
hisrng = np.random.default_rng(seed=42)
herrng = np.random.default_rng(seed=24)

# Now we can draw some numbers by applying the Generator to the BitGenerator
hisvals = hisrng.standard_normal(5)
hervals = herrng.standard_normal(5)
print(f"His random numbers: \n{hisvals}")
print(f"Her random numbers: \n{hervals}")

# We can reseed one BitGenerator without affecting the other
newvals = rng.standard_normal(5)
print(f"New random numbers: \n{newvals}")
# Now we can reseed the BitGenerator
rng = np.random.default_rng(seed=42)
newvals = rng.standard_normal(5)
print(f"New random numbers: \n{newvals}")

His random numbers: 
[ 0.30471708 -1.03998411  0.7504512   0.94056472 -1.95103519]
Her random numbers: 
[ 1.35074732  0.34309058 -1.16299016 -0.18708582 -0.3394646 ]
New random numbers: 
[-1.10978358 -0.78537895  1.93033735 -0.06741104 -0.89120049]
New random numbers: 
[ 0.30471708 -1.03998411  0.7504512   0.94056472 -1.95103519]


***

## Shuffling and Bootstrapping

We often want to draw __entries__ from sequences or arrays or series or dataframes. <br>
Python and NumPy have _functions_ for that, and Pandas has _methods_.

#### Draw ``MyNum`` elements (with replacement) from some variable ``MyVar``.
- **Python:** ``choices(MyVar,k=MyNum)``<br>
- **NumPy:**  ``choice(MyVar,size=MyNum)``<br>
- **Pandas:** ``MyVar.sample(n=MyNum,replace=True)``

#### Draw ``MyNum`` elements (without replacement) from some variable ``MyVar``.
- **Python:** ``sample(MyVar,k=MyNum)``<br>
- **NumPy:**  ``choice(MyVar,size=MyNum,replace=False)``<br>
- **Pandas:** ``MyVar.sample(n=MyNum)``

#### "Shuffle": randomly reorder the elements in ``MyVar``.
- **Python:** ``shuffle(MyVar)``<br>
- **NumPy:**  ``shuffle(MyVar)``<br>
- **Pandas:** ``MyVar.sample(frac=1).reset_index(drop=True)``

All three functions shuffle the series _in-place_.<br>
NumPy also has ``permutation(MyVar)``, which returns a shuffled _copy_ of ``MyVar``.

Note that all of these commands have additional options (e.g. for weighted sampling.) 
- NumPy's ``choice()`` only works with vectors (1-D arrays, not 2-D.) <br>
  (A newer syntax using ``random.Generator`` can be used for 2-D arrays.) <br>
  ``shuffle`` works with higher dimensional arrays, but only shuffles axis 0 (i.e. rows.) 
- The Pandas functions allow the user to specify the axis.<br>
  By default, rows are sampled, but columns may be sampled instead.


### Examples !

In [28]:
import numpy.random as npr
import pandas as pd

NumColumns = 3
NumObs = 10

My_seed_for_the_generator = 19980311
npr.seed(My_seed_for_the_generator)

np_junk = npr.rand(NumObs,NumColumns)
print(f'The NumPy array contains \n{np_junk}.')

The NumPy array contains 
[[9.09678224e-04 8.48252816e-01 4.28905309e-01]
 [1.18630181e-01 7.00499703e-02 6.34002815e-01]
 [8.30516270e-01 3.78093782e-01 9.66343091e-01]
 [4.96996147e-01 5.15582686e-02 3.92745535e-01]
 [9.02364532e-01 8.04527617e-01 7.68698620e-01]
 [5.97651739e-01 4.30511805e-01 4.85653631e-01]
 [6.71777372e-01 6.37578665e-01 4.91498642e-01]
 [1.25089165e-01 9.98365708e-01 6.92263457e-02]
 [5.24001415e-01 2.07647847e-01 5.38478967e-01]
 [7.63488458e-01 2.83294068e-01 8.94343098e-01]].


In [29]:
pd_junk = pd.DataFrame(np_junk)
print(f'The Pandas dataframe contains \n{pd_junk}')

The Pandas dataframe contains 
          0         1         2
0  0.000910  0.848253  0.428905
1  0.118630  0.070050  0.634003
2  0.830516  0.378094  0.966343
3  0.496996  0.051558  0.392746
4  0.902365  0.804528  0.768699
5  0.597652  0.430512  0.485654
6  0.671777  0.637579  0.491499
7  0.125089  0.998366  0.069226
8  0.524001  0.207648  0.538479
9  0.763488  0.283294  0.894343


In [30]:
MyNum = 5

# Draw without replacement
print(f'NumPy:  Drawing {MyNum} observations without replacement from the first column gives \n {npr.choice(np_junk[:,0],size=MyNum,replace=False)}.')
print(f'Pandas: Drawing {MyNum} observations without replacement from the first column gives \n {pd_junk.sample(n=MyNum)[0]}.')

NumPy:  Drawing 5 observations without replacement from the first column gives 
 [0.90236453 0.67177737 0.76348846 0.49699615 0.11863018].
Pandas: Drawing 5 observations without replacement from the first column gives 
 7    0.125089
3    0.496996
2    0.830516
1    0.118630
9    0.763488
Name: 0, dtype: float64.


In [31]:
# Draw with replacement
print(f'NumPy:  Drawing {MyNum} observations with replacement from the first column replacement gives \n {npr.choice(np_junk[:,0],size=MyNum)}.')
print(f'Pandas: Drawing {MyNum} observations with replacement from the first column replacement gives \n {pd_junk.sample(n=MyNum,replace=True)[0]}.')

NumPy:  Drawing 5 observations with replacement from the first column replacement gives 
 [0.90236453 0.52400142 0.00090968 0.76348846 0.59765174].
Pandas: Drawing 5 observations with replacement from the first column replacement gives 
 6    0.671777
4    0.902365
2    0.830516
3    0.496996
3    0.496996
Name: 0, dtype: float64.


In [32]:
# Pandas allows us to sample rows or columns.
print( pd_junk.sample(n=MyNum) )
print( pd_junk.sample(n=MyNum,axis=1,replace=True) )

          0         1         2
4  0.902365  0.804528  0.768699
9  0.763488  0.283294  0.894343
0  0.000910  0.848253  0.428905
3  0.496996  0.051558  0.392746
1  0.118630  0.070050  0.634003
          1         0         0         2         1
0  0.848253  0.000910  0.000910  0.428905  0.848253
1  0.070050  0.118630  0.118630  0.634003  0.070050
2  0.378094  0.830516  0.830516  0.966343  0.378094
3  0.051558  0.496996  0.496996  0.392746  0.051558
4  0.804528  0.902365  0.902365  0.768699  0.804528
5  0.430512  0.597652  0.597652  0.485654  0.430512
6  0.637579  0.671777  0.671777  0.491499  0.637579
7  0.998366  0.125089  0.125089  0.069226  0.998366
8  0.207648  0.524001  0.524001  0.538479  0.207648
9  0.283294  0.763488  0.763488  0.894343  0.283294


In [33]:
# Pandas will also allow us to seed its RNG
print( pd_junk.sample(n=MyNum, random_state = My_seed_for_the_generator) )
print( pd_junk.sample(n=MyNum, random_state = My_seed_for_the_generator) )

          0         1         2
7  0.125089  0.998366  0.069226
0  0.000910  0.848253  0.428905
6  0.671777  0.637579  0.491499
2  0.830516  0.378094  0.966343
5  0.597652  0.430512  0.485654
          0         1         2
7  0.125089  0.998366  0.069226
0  0.000910  0.848253  0.428905
6  0.671777  0.637579  0.491499
2  0.830516  0.378094  0.966343
5  0.597652  0.430512  0.485654


In [34]:
# Permutations
print(f"Here's the original array: \n {np_junk}")
npr.seed(My_seed_for_the_generator)
print(f"Here's the permuted array: \n {npr.permutation(np_junk)}")
npr.seed(My_seed_for_the_generator)
npr.shuffle(np_junk)
print(f"Here's the shuffled array: \n {np_junk}")
# Note that the shuffled and permuted arrays are the same!
npr.seed(My_seed_for_the_generator)
npr.shuffle(np_junk)
print(f"Here's the shuffled array: \n {np_junk}")

Here's the original array: 
 [[9.09678224e-04 8.48252816e-01 4.28905309e-01]
 [1.18630181e-01 7.00499703e-02 6.34002815e-01]
 [8.30516270e-01 3.78093782e-01 9.66343091e-01]
 [4.96996147e-01 5.15582686e-02 3.92745535e-01]
 [9.02364532e-01 8.04527617e-01 7.68698620e-01]
 [5.97651739e-01 4.30511805e-01 4.85653631e-01]
 [6.71777372e-01 6.37578665e-01 4.91498642e-01]
 [1.25089165e-01 9.98365708e-01 6.92263457e-02]
 [5.24001415e-01 2.07647847e-01 5.38478967e-01]
 [7.63488458e-01 2.83294068e-01 8.94343098e-01]]
Here's the permuted array: 
 [[1.25089165e-01 9.98365708e-01 6.92263457e-02]
 [9.09678224e-04 8.48252816e-01 4.28905309e-01]
 [6.71777372e-01 6.37578665e-01 4.91498642e-01]
 [8.30516270e-01 3.78093782e-01 9.66343091e-01]
 [5.97651739e-01 4.30511805e-01 4.85653631e-01]
 [1.18630181e-01 7.00499703e-02 6.34002815e-01]
 [7.63488458e-01 2.83294068e-01 8.94343098e-01]
 [5.24001415e-01 2.07647847e-01 5.38478967e-01]
 [9.02364532e-01 8.04527617e-01 7.68698620e-01]
 [4.96996147e-01 5.15582686e-

***

## Distributions

As we saw above, ``random()`` and ``rand()`` generate random values $\sim U(0,1)$. Often we want random numbers draw from different distributions. 

**Python's** ``random`` module includes such functions for a number of such distributions, such as 

- ``gauss``(mu,sigma)
- ``lognormvariate``(mu,sigma)
- ``uniform``(a,b)
- ``betavariate``(alpha,beta)
- ``gammavariate``(alpha, beta)

**NumPy** also has a variety of similar functions which share the parameter ``size``, a tuple (or similar) giving the shape of the output.

- ``normal``(loc=mu, scale=sigma, size)
- ``beta``(alpha, beta, size)
- ``binomial``(n,p,size)
- ``chisquare``(df,size)
- ``gamma``(shape,scale,size)


---
## Useful Tricks

We can use the features that we've seen so far with some theory to do some handy tricks!

### Drawing from an Unconventional Distribution

What if we want to draw random values from a distribution that isn't included in Python or NumPy?<br>
We can draw random values from *any* distribution for which we know the inverse cdf $F^{-1}(x)$. <br>
To do so, 
1. program $F^{-1}(x)$; let's assume we called it ``Finv()``.
2. Simply call ``Finv(rand())`` or ``Finv(random())``.

This interprets random values between 0 and 1 as _probabilities_, and then maps them back into distribution that we want. <br>
(This is deeply related to PIT tests, which we'll talk about later in the course.) 

### Drawing a Dummy Variable

One of the simplest tasks is drawing a **dummy** (a.k.a. indicator, binary, Boolean, dichotomous, 0/1, etc.) variable.

Let ``MyDummy`` be the variable we're creating and let ``pDummy`` be the probability that it is equal to 1.

In [35]:
# Dummies
pDummy = 0.7
MyDummy = npr.rand(10) < pDummy
print(f"Here's my dummy variable: \n {MyDummy} \n {MyDummy*1}")

Here's my dummy variable: 
 [False  True  True  True  True  True  True False  True  True] 
 [0 1 1 1 1 1 1 0 1 1]


In [36]:
Dsize = 20
pDummy = npr.rand(Dsize)
MyDummy = npr.rand(Dsize) < pDummy

from numpy import vstack

print(f"Dummies with variable probabilities: \n {vstack((MyDummy,pDummy)).T}" )

Dummies with variable probabilities: 
 [[0.         0.31023503]
 [1.         0.97051137]
 [1.         0.5898573 ]
 [0.         0.89889785]
 [0.         0.04516918]
 [0.         0.46858702]
 [0.         0.00141648]
 [1.         0.84176875]
 [1.         0.44545046]
 [1.         0.26846803]
 [1.         0.4878885 ]
 [0.         0.16778074]
 [0.         0.17727023]
 [1.         0.6941738 ]
 [0.         0.53782587]
 [0.         0.27708016]
 [0.         0.09292731]
 [1.         0.48324252]
 [0.         0.21335464]
 [0.         0.0559934 ]]


### Correlated Variables

All of the methods shown above take *independent* draws from a distribution. <br>
But suppose we want to draw several random variables that are correlated with one another?

If we want to draw from a **normal** distribution, we can use Numpy's ``multivariate_normal()`` function.

*Syntax:* **random.multivariate_normal(mean, cov, size=None)**<br>
where
- **mean** is a 1-D array-like of means (dimension ``N``)
- **cov** is a 2-D array-like of covariance matrix (``N``x``N`` )<br>Should be positive-semi-definite and symmetric.
- **size** is an int (or tuple of ints) giving the number of draws of each series.

Alternatively, we can use the fact that, if we have a vector of variables $Y$ where $V(Y) = I$<br>
then $V(Y \cdot D) = D^\prime \cdot I \cdot D = D^\prime \cdot D$. 
That means that we can 
- generate $N$ identical i.i.d. variables ``Yiid`` (so $V(Yiid) = I$).
- choose $D$ such that $D^\prime \cdot D = \Sigma$ where $\Sigma$ is the VCV matrix we want.<br>
	(A Choleski decomposition of $\Sigma$ is one way to get $D$.)
- Create ``Ycorr = Yiid @ D`` to get our correlated variables.

Note that the second method works for *any* distribution.